In [43]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-lg61k0gb/gym-walk_d509725f3df9480fb19cfc9d33bd51c4
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-lg61k0gb/gym-walk_d509725f3df9480fb19cfc9d33bd51c4
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done


In [44]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

# Reference https://github.com/mimoralea/gym-walk

In [45]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [46]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [47]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [48]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [49]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
LEFT, DOWN, RIGHT, UP = range(4)

In [50]:

P


{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [51]:
init_state

0

In [53]:
state, reward, done, info = env.step(RIGHT)
print("state:{0} - reward:{1} - done:{2} - info:{3}".format(state, reward, done, info))


state:4 - reward:0.0 - done:False - info:{'prob': 0.3333333333333333}


In [54]:

pi_1 = lambda s: {
    0: RIGHT,
    1: DOWN,
    2: RIGHT,
    3: LEFT,
    4: DOWN,
    5: LEFT,
    6: RIGHT,
    7:LEFT,
    8: UP,
    9: DOWN,
    10:LEFT,
    11:DOWN,
    12:RIGHT,
    13:RIGHT,
    14:DOWN,
    15:LEFT #Stop
}[s]
print_policy(pi_1, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Policy:
| 00      > | 01      v | 02      > | 03      < |
| 04      v |           | 06      > |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [55]:

print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_1, goal_state=goal_state) * 100,
    mean_return(env, pi_1)))

Reaches goal 10.00%. Obtains an average undiscounted return of 0.1000.


In [57]:
pi_2 = lambda s: {
    0: DOWN,
    1: RIGHT,
    2: DOWN,
    3: LEFT,
    4: UP,
    5: RIGHT,
    6: UP,
    7: LEFT,
    8: RIGHT,
    9: UP,
    10: LEFT,
    11: DOWN,
    12: RIGHT,
    13: UP,
    14: UP,
    15: RIGHT
}[s]
print_policy(pi_2, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)
print("Name:Iyyanar S")
print("Reg no:212222240036")


Policy:
| 00      v | 01      > | 02      v | 03      < |
| 04      ^ |           | 06      ^ |           |
| 08      > | 09      ^ | 10      < |           |
|           | 13      ^ | 14      ^ |           |
Name:Iyyanar S
Reg no:212222240036


In [58]:
# Find the probability of success and the mean return of you your policy
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_2, goal_state=goal_state)*100,
    mean_return(env, pi_2)))

Reaches goal 0.00%. Obtains an average undiscounted return of 0.0000.


In [59]:
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    prev_V = np.zeros(len(P), dtype=np.float64)
    # Write your code here to evaluate the given policy
    while True:
      V = np.zeros(len(P))
      for s in range(len(P)):
        for prob, next_state, reward, done in P[s][pi(s)]:
          V[s] += prob * (reward + gamma *  prev_V[next_state] * (not done))
      if np.max(np.abs(prev_V - V)) < theta:
        break
      prev_V = V.copy()
    return V

In [60]:
# Code to evaluate the first policy
V1 = policy_evaluation(pi_1, P)
print_state_value_function(V1, P, n_cols=7, prec=5)

State-value function:
| 00 0.13333 | 01 0.09462 | 02 0.15054 | 03 0.07527 | 04 0.17204 |           | 06 0.22581 |
|           | 08 0.34409 | 09 0.51613 | 10 0.52688 |           |           | 13 0.67742 |
| 14 0.83871 |           

In [61]:
# Code to evaluate the second policy
V2 = policy_evaluation(pi_2, P)
print_state_value_function(V2, P, n_cols=7, prec=5)

State-value function:
| 00    0.0 | 01    0.0 | 02    0.0 | 03    0.0 | 04    0.0 |           | 06    0.0 |
|           | 08 0.02128 | 09 0.06383 | 10 0.17021 |           |           | 13 0.17021 |
| 14 0.44681 |           

In [62]:
V1

array([0.13333333, 0.09462366, 0.15053763, 0.07526882, 0.17204301,
       0.        , 0.22580645, 0.        , 0.34408602, 0.51612903,
       0.52688172, 0.        , 0.        , 0.67741935, 0.83870968,
       0.        ])

In [63]:
print_state_value_function(V1, P, n_cols=7, prec=5)

State-value function:
| 00 0.13333 | 01 0.09462 | 02 0.15054 | 03 0.07527 | 04 0.17204 |           | 06 0.22581 |
|           | 08 0.34409 | 09 0.51613 | 10 0.52688 |           |           | 13 0.67742 |
| 14 0.83871 |           

In [64]:
V2

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.0212766 , 0.06382979,
       0.17021277, 0.        , 0.        , 0.17021277, 0.44680851,
       0.        ])

In [65]:
print_state_value_function(V2, P, n_cols=7, prec=5)

State-value function:
| 00    0.0 | 01    0.0 | 02    0.0 | 03    0.0 | 04    0.0 |           | 06    0.0 |
|           | 08 0.02128 | 09 0.06383 | 10 0.17021 |           |           | 13 0.17021 |
| 14 0.44681 |           

In [66]:
V1>=V2

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [67]:
if(np.sum(V1>=V2)==7):
  print("The first policy is the better policy")
elif(np.sum(V2>=V1)==7):
  print("The second policy is the better policy")
else:
  print("Both policies have their merits.")

Both policies have their merits.
